# Document Chat Bot with Knowledge Graph & ReAct Agent

This notebook provides an interactive chat interface for querying multiple documents (PDF, JSON, JSONL, TXT) with Knowledge Graph enhancement and optional ReAct Agent for complex reasoning.

## Quick Start Workflow

1. **Run cells 1-4**: Install packages, import libraries, configure settings
2. **Run cell 9**: Upload your documents and extract text
3. **Run cell 15**: Initialize Knowledge Graph and chat interface
4. **Run cell 17**: Start continuous chat - ask unlimited questions until you type `exit()`
5. **Run cell 19** (optional): View full chat history

## Features

- **Multi-format support**: PDF, JSON, JSONL, and TXT files
- **Knowledge Graph**: Automatically extracts entities (controls, risks, assets, etc.) and relationships
- **ReAct Agent**: Advanced multi-step reasoning for complex queries with automatic routing
- **Continuous chat**: Ask multiple questions in one session
- **Chat history**: View all questions and answers
- **Enhanced responses**: Better context through entity and relationship tracking

## ReAct Agent with Intelligent Routing

The **ReAct Agent** uses your LLM iteratively with specialized tools for complex multi-step reasoning. The agent **automatically activates** for complex queries!

### How Agent-LLM Interaction Works

```
USER QUERY: "What would be impacted if we remove AC-2?"
    ↓
┌─────────────────────────────────────┐
│ LLM CALL #1: Planning               │
│ "I need to find AC-2 first"         │
│ → Decides to use search_entities    │
└─────────────────────────────────────┘
    ↓
┌─────────────────────────────────────┐
│ TOOL: search_entities("AC-2")       │
│ → Returns: CONTROL_AC-2 found       │
└─────────────────────────────────────┘
    ↓
┌─────────────────────────────────────┐
│ LLM CALL #2: Interpret              │
│ "Found AC-2. Now get relationships" │
│ → Decides to use get_relationships  │
└─────────────────────────────────────┘
    ↓
┌─────────────────────────────────────┐
│ TOOL: get_entity_relationships      │
│ → Returns: Connected entities       │
└─────────────────────────────────────┘
    ↓
┌─────────────────────────────────────┐
│ LLM CALL #3: Continue               │
│ "Need downstream impacts"           │
│ → Decides to use traverse_graph     │
└─────────────────────────────────────┘
    ↓
┌─────────────────────────────────────┐
│ TOOL: traverse_graph(depth=2)       │
│ → Returns: Full dependency tree     │
└─────────────────────────────────────┘
    ↓
┌─────────────────────────────────────┐
│ LLM CALL #4: Synthesize             │
│ "I have everything. Final answer:"  │
│ → Generates comprehensive report    │
└─────────────────────────────────────┘
    ↓
COMPREHENSIVE RESPONSE
```

**Key Points:**
- **Same LLM** used throughout (4-10+ calls per complex query)
- **Agent = LLM + Tools + Iterative Reasoning**
- Each cycle: Plan → Execute Tool → Interpret Results → Repeat
- Final LLM call synthesizes all gathered information

**Comparison:**
- **Simple Flow:** 1 LLM call (fast, direct answer)
- **Agent Flow:** 4-10+ LLM calls (thorough, multi-step reasoning)

**How It Works in This Notebook:**
- Configure agent settings in the configuration cell (Cell 5)
- Agent automatically activates when query complexity exceeds threshold (60/100)
- Simple queries use direct KG retrieval for speed
- Complex queries use multi-step agent reasoning for thoroughness

## Commands During Chat

- Type your question and press Enter
- Type `exit()` or `quit()` to end the chat session
- Type `history` to view chat history
- Press Ctrl+C to interrupt (then type exit() to quit properly)


## 1. Install Required Packages


In [ ]:
%pip install goldmansachs.awm_genai -U
%pip install python-dotenv pandas ipywidgets pdfplumber networkx matplotlib -q
%pip install langgraph langchain langchain-core -q


## 2. Import Libraries and Configuration


In [ ]:
from goldmansachs.awm_genai import LLM, LLMConfig
import os
from typing import List, Dict
import pandas as pd
from datetime import datetime
import tempfile
from IPython.display import display, HTML
import ipywidgets as widgets
import pdfplumber
import json
import re


In [ ]:
# Configuration
app_id = "trai"
env = "uat"

# Model Configuration - Choose your model
available_models = ["gemini-2.5-pro", "gemini-2.5-flash-lite"]

# Create model selection widget
model_selector = widgets.Dropdown(
    options=available_models,
    value="gemini-2.5-flash-lite",
    description='Model:',
    style={'description_width': 'initial'}
)

# Agent Configuration - NEW!
enable_agent_toggle = widgets.Checkbox(
    value=True,
    description='Enable ReAct Agent Mode',
    style={'description_width': 'initial'}
)

show_reasoning_toggle = widgets.Checkbox(
    value=True,
    description='Show Agent Reasoning',
    style={'description_width': 'initial'}
)

# Display configuration
display(HTML("<h3>Configuration</h3>"))
display(HTML("<p><b>gemini-2.5-pro:</b> More capable, better for complex questions<br><b>gemini-2.5-flash-lite:</b> Faster responses, good for simple queries</p>"))
display(model_selector)

display(HTML("<br><h4>Agent Settings</h4>"))
display(HTML("<p>Enable agent mode for advanced multi-step reasoning on complex queries. The agent uses your LLM iteratively with specialized tools.</p>"))
display(enable_agent_toggle)
display(show_reasoning_toggle)

# Vespa Configuration
display(HTML("<br><h4>Vespa Vector Store</h4>"))
display(HTML("<p>Enable Vespa search for additional context. Used as fallback when no documents uploaded.</p>"))

enable_vespa_toggle = widgets.Checkbox(
    value=True,
    description='Enable Vespa Search',
    style={'description_width': 'initial'}
)

vespa_schema_input = widgets.Text(
    value='tech_risk_ai',
    description='Schema ID:',
    style={'description_width': 'initial'}
)

vespa_env_selector = widgets.Dropdown(
    options=['dev', 'uat', 'prod'],
    value='uat',
    description='Vespa Env:',
    style={'description_width': 'initial'}
)

vespa_gssso_input = widgets.Text(
    value='',
    placeholder='Enter GSSSO token (optional)',
    description='GSSSO Token:',
    style={'description_width': 'initial'}
)

vespa_api_key_input = widgets.Text(
    value='',
    placeholder='Enter API key (optional)',
    description='API Key:',
    style={'description_width': 'initial'}
)

display(enable_vespa_toggle)
display(vespa_schema_input)
display(vespa_env_selector)
display(HTML("<p><small><b>Authentication (Optional):</b> Add GSSSO token or API key if getting 401/500 errors</small></p>"))
display(vespa_gssso_input)
display(vespa_api_key_input)

# Store configuration
temperature = 0
log_level = "DEBUG"
agent_temperature = 0.2
agent_max_iterations = 10
agent_complexity_threshold = 60

print(f"\nApp ID: {app_id}")
print(f"Environment: {env}")
print(f"Agent Mode: {'Enabled' if enable_agent_toggle.value else 'Disabled'}")
print(f"Vespa Search: {'Enabled' if enable_vespa_toggle.value else 'Disabled'}")


## 3. Initialize LLM


In [ ]:
# Initialize LLM with selected model
model_name = model_selector.value

llm_config = LLMConfig(
    app_id=app_id,
    env=env,
    model_name=model_name,
    temperature=temperature,
    log_level=log_level,
)

llm = LLM.init(config=llm_config)
print(f"[SUCCESS] LLM initialized successfully with {model_name}")


## 4. Upload Documents

Use the file upload widget below to select your PDF documents.


In [ ]:
# Helper function to extract PDF content with tables and JSON
def extract_pdf_content(file_path: str, filename: str) -> str:
    """Extract text, tables, and JSON from PDF."""
    content_parts = [f"\n\n{'='*80}\nDocument: {filename}\n{'='*80}\n"]
    
    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages, 1):
            content_parts.append(f"\n[Page {page_num}]\n")
            
            # Extract tables on this page
            tables = page.extract_tables()
            
            # Get bounding boxes of tables to exclude from text
            table_bboxes = []
            if tables:
                for table in page.find_tables():
                    table_bboxes.append(table.bbox)
            
            # Extract text excluding table areas
            if table_bboxes:
                text = page.filter(lambda obj: not any(
                    obj.get('x0', 0) >= bbox[0] and obj.get('x1', 0) <= bbox[2] and
                    obj.get('top', 0) >= bbox[1] and obj.get('bottom', 0) <= bbox[3]
                    for bbox in table_bboxes
                )).extract_text()
            else:
                text = page.extract_text()
            
            # Check for JSON/JSONL content in text
            if text and text.strip():
                json_objects = extract_json_content(text)
                
                if json_objects:
                    # Add regular text (non-JSON parts)
                    non_json_text = remove_json_from_text(text)
                    if non_json_text.strip():
                        content_parts.append(f"{non_json_text}\n")
                    
                    # Add formatted JSON objects
                    for json_idx, json_obj in enumerate(json_objects, 1):
                        formatted_json = format_json_object(json_obj, page_num, json_idx, filename)
                        content_parts.append(f"\n{formatted_json}\n")
                else:
                    # No JSON, add as regular text
                    content_parts.append(f"{text}\n")
            
            # Add tables with proper formatting
            if tables:
                for table_idx, table in enumerate(tables, 1):
                    if table and len(table) > 0:
                        formatted_table = format_table(table, page_num, table_idx, filename)
                        content_parts.append(f"\n{formatted_table}\n")
    
    return "\n".join(content_parts)

def extract_json_content(text: str) -> list:
    """Extract JSON or JSONL objects from text."""
    json_objects = []
    
    # Try to find JSON objects
    json_pattern = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
    matches = re.finditer(json_pattern, text, re.DOTALL)
    
    for match in matches:
        try:
            json_str = match.group(0)
            json_obj = json.loads(json_str)
            json_objects.append(json_obj)
        except json.JSONDecodeError:
            pass
    
    # Also try line-by-line for JSONL format
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('{') and line.endswith('}'):
            try:
                json_obj = json.loads(line)
                if json_obj not in json_objects:
                    json_objects.append(json_obj)
            except:
                pass
    
    return json_objects

def remove_json_from_text(text: str) -> str:
    """Remove JSON objects from text to get only regular text."""
    json_pattern = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
    cleaned_text = re.sub(json_pattern, '', text, flags=re.DOTALL)
    return cleaned_text

def format_json_object(json_obj: dict, page_num: int, json_idx: int, filename: str) -> str:
    """Format JSON object for LLM understanding."""
    json_parts = []
    json_parts.append(f"--- JSON OBJECT {json_idx} (Document: {filename}, Page {page_num}) ---")
    
    # Add formatted key-value pairs
    json_parts.append("\nStructured Data Fields:")
    for key, value in json_obj.items():
        # Clean up the value
        if isinstance(value, str):
            value = ' '.join(value.split())
        json_parts.append(f"  {key}: {value}")
    
    # Add JSON format
    json_parts.append("\nJSON Format:")
    json_parts.append(json.dumps(json_obj, indent=2))
    
    json_parts.append(f"--- END JSON OBJECT {json_idx} ---\n")
    
    return "\n".join(json_parts)

def format_table(table: list, page_num: int, table_idx: int, filename: str) -> str:
    """Format table with proper structure."""
    if not table or len(table) == 0:
        return ""
    
    # Clean table data
    cleaned_table = []
    for row in table:
        cleaned_row = [str(cell).strip() if cell is not None else "" for cell in row]
        if any(cleaned_row):
            cleaned_table.append(cleaned_row)
    
    if not cleaned_table:
        return ""
    
    table_parts = []
    table_parts.append(f"--- TABLE {table_idx} (Document: {filename}, Page {page_num}) ---")
    
    # Assume first row is header
    headers = cleaned_table[0]
    data_rows = cleaned_table[1:]
    
    # Add headers
    table_parts.append("\nColumn Headers:")
    table_parts.append(" | ".join(headers))
    table_parts.append("-" * 80)
    
    # Add data rows
    table_parts.append("\nTable Data:")
    for row in data_rows:
        table_parts.append(" | ".join(row))
    
    # Add markdown format for better LLM understanding
    table_parts.append("\nMarkdown Format:")
    table_parts.append("| " + " | ".join(headers) + " |")
    table_parts.append("|" + "|".join(["---" for _ in headers]) + "|")
    for row in data_rows:
        table_parts.append("| " + " | ".join(row) + " |")
    
    table_parts.append(f"--- END TABLE {table_idx} ---\n")
    
    return "\n".join(table_parts)

# Helper functions for JSON extraction
def extract_from_json_file(file_path: str, filename: str) -> str:
    """Extract and format JSON file content."""
    content_parts = [f"\n\n{'='*80}\nDocument: {filename}\n{'='*80}\n"]
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if isinstance(data, list):
            content_parts.append("\nThis file contains a list of JSON objects:\n")
            for idx, obj in enumerate(data, 1):
                formatted = format_json_object(obj, 0, idx, filename)
                content_parts.append(f"\n{formatted}\n")
        elif isinstance(data, dict):
            formatted = format_json_object(data, 0, 1, filename)
            content_parts.append(f"\n{formatted}\n")
        else:
            content_parts.append(f"\nJSON Value: {data}\n")
    except Exception as e:
        content_parts.append(f"\n[ERROR] Failed to parse JSON: {str(e)}\n")
    
    return "\n".join(content_parts)

def extract_from_jsonl_file(file_path: str, filename: str) -> str:
    """Extract and format JSONL file content."""
    content_parts = [f"\n\n{'='*80}\nDocument: {filename}\n{'='*80}\n"]
    content_parts.append("\nThis file contains multiple JSON objects (one per line):\n")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for idx, line in enumerate(f, 1):
                line = line.strip()
                if line:
                    try:
                        obj = json.loads(line)
                        formatted = format_json_object(obj, 0, idx, filename)
                        content_parts.append(f"\n{formatted}\n")
                    except json.JSONDecodeError:
                        content_parts.append(f"\n[Line {idx}] Invalid JSON: {line[:100]}...\n")
    except Exception as e:
        content_parts.append(f"\n[ERROR] Failed to parse JSONL: {str(e)}\n")
    
    return "\n".join(content_parts)

# Create file upload widget
upload_widget = widgets.FileUpload(
    accept='.pdf,.json,.jsonl,.txt',
    multiple=True,
    description='Select Files'
)

# Create process button
process_button = widgets.Button(
    description='Extract Text',
    button_style='primary',
    icon='check'
)

# Create output widget for status messages
output = widgets.Output()

# Store extracted text globally
extracted_text = None
document_names = []

def on_process_button_clicked(b):
    global extracted_text, document_names
    
    with output:
        output.clear_output()
        
        if not upload_widget.value:
            print("[WARNING] Please select PDF files first")
            return
        
        try:
            all_text = []
            document_names = []
            
            # Extract content from uploaded files
            files = upload_widget.value
            print(f"Processing {len(files)} files...\n")
            
            temp_files = []
            document_names = []
            
            # Save uploaded files temporarily
            for file_info in files:
                filename = file_info['name']
                content = file_info['content']
                document_names.append(filename)
                
                with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
                    tmp_file.write(content)
                    temp_files.append((tmp_file.name, filename))
            
            # Extract content based on file type
            for tmp_path, filename in temp_files:
                try:
                    file_ext = filename.split('.')[-1].lower()
                    
                    if file_ext == 'pdf':
                        doc_content = extract_pdf_content(tmp_path, filename)
                    elif file_ext == 'json':
                        doc_content = extract_from_json_file(tmp_path, filename)
                    elif file_ext == 'jsonl':
                        doc_content = extract_from_jsonl_file(tmp_path, filename)
                    elif file_ext == 'txt':
                        # Extract from text file
                        with open(tmp_path, 'r', encoding='utf-8') as f:
                            text_content = f.read()
                        doc_content = f"\n\n{'='*80}\nDocument: {filename}\n{'='*80}\n\n{text_content}\n"
                    else:
                        doc_content = f"\n[ERROR] Unsupported file type: {file_ext}\n"
                    
                    all_text.append(doc_content)
                    print(f"  [OK] {filename} - extracted successfully")
                finally:
                    os.unlink(tmp_path)
            
            # Combine all extracted text
            extracted_text = "\n\n".join(all_text)
            
            print(f"\n[SUCCESS] Successfully extracted text from {len(files)} documents")
            print(f"Total characters: {len(extracted_text):,}")
                
        except Exception as e:
            print(f"[ERROR] Error: {str(e)}")
            import traceback
            traceback.print_exc()

process_button.on_click(on_process_button_clicked)

# Display widgets
display(HTML("<h3>Upload and Extract Documents</h3>"))
display(HTML("<p>Supported formats: PDF, JSON, JSONL, TXT</p>"))
display(upload_widget)
display(process_button)
display(output)

print("Use the widget above to select files (PDF, JSON, JSONL, or TXT) and extract their content")


## 5. Ask Questions


In [ ]:
# This cell prepares the system - no action needed, just run it
if extracted_text is None:
    print("[WARNING] No documents uploaded yet")
    print("You can either:")
    print("  1. Upload documents in section 4")
    print("  2. OR skip to section 7 to use Vespa only")
else:
    print("[SUCCESS] Documents ready!")
    print(f"Documents loaded: {', '.join(document_names)}")
    print(f"Total content: {len(extracted_text):,} characters")
    print("\n[NEXT STEP] Run section 7 (cell 15) to initialize chat interface")


## 6. Initialize Knowledge Graph & Chat Interface

Run this section to build the Knowledge Graph and initialize the chat system.


In [ ]:
# This section is now handled in section 7 - skip to section 7
print("Skip to section 7 to initialize the chat interface with Knowledge Graph.")


## 7. Initialize Chat Interface with Knowledge Graph

Run this cell to build the Knowledge Graph and prepare the chat system. This only needs to be run once after uploading documents.


In [ ]:
# Initialize Knowledge Graph (if documents available)
use_kg = True
kg_retriever = None

if extracted_text and use_kg:
    try:
        from kg_retriever import KGRetriever
        
        print("Building Knowledge Graph for enhanced responses...")
        
        # Prepare documents for KG
        documents_for_kg = []
        for name in document_names:
            documents_for_kg.append({
                'name': name,
                'content': extracted_text
            })
        
        # Build KG
        kg_retriever = KGRetriever()
        kg_retriever.build_knowledge_graph(documents_for_kg)
        
        stats = kg_retriever.get_statistics()
        print(f" Knowledge Graph built: {stats['entity_count']} entities, {stats['relationship_count']} relationships\n")
    except ImportError:
        print(" Knowledge Graph module not found. Using standard chat mode.\n")
        use_kg = False
    except Exception as e:
        print(f" Could not build Knowledge Graph: {str(e)}")
        print("Using standard chat mode.\n")
        use_kg = False
elif not extracted_text:
    print("No documents uploaded - will use Vespa for context if enabled.\n")
    use_kg = False

# Initialize Vespa Vector Store (works with or without documents)
vespa_wrapper = None

if enable_vespa_toggle.value:
    try:
        from vespa_search import create_vespa_wrapper
        
        print("Connecting to Vespa Vector Store...")
        
        # Get auth parameters if provided
        gssso_token = vespa_gssso_input.value if vespa_gssso_input.value else None
        api_key = vespa_api_key_input.value if vespa_api_key_input.value else None
        
        vespa_wrapper = create_vespa_wrapper(
            schema_id=vespa_schema_input.value,
            env=vespa_env_selector.value,
            gssso_token=gssso_token,
            api_key=api_key
        )
        
        if vespa_wrapper and vespa_wrapper.is_available():
            # Test connection
            test_result = vespa_wrapper.test_connection()
            
            if test_result.get('success'):
                print(f" Vespa connected: {vespa_schema_input.value} ({vespa_env_selector.value})")
                if gssso_token or api_key:
                    print(f" Auth: Using {'GSSO token' if gssso_token else ''} {'+ API key' if api_key else ''}")
                print()
            else:
                print(f" Vespa connection test failed:")
                print(f" Error: {test_result.get('error')}")
                if test_result.get('suggestion'):
                    print(f" Suggestion: {test_result['suggestion']}")
                print()
                vespa_wrapper = None
        else:
            print(" Vespa connection failed - will use documents only\n")
            vespa_wrapper = None
    except Exception as e:
        print(f" Could not connect to Vespa: {str(e)}")
        print(" Will use documents only\n")
        vespa_wrapper = None

# Initialize ReAct Agent (works with or without documents if Vespa available)
agent_orchestrator = None
query_router = None
agent_state = None

# Create minimal KG for agent even without documents
if enable_agent_toggle.value and not kg_retriever:
    from kg_retriever import KGRetriever
    kg_retriever = KGRetriever()
    print("Created minimal KG for agent (will use Vespa for context)\n")

if enable_agent_toggle.value and kg_retriever:
    try:
        from query_router import QueryRouter
        from react_agent import AgentOrchestrator
        from agent_state import AgentState
        
        print("Initializing ReAct Agent...")
        
        # Create query router
        query_router = QueryRouter(complexity_threshold=agent_complexity_threshold)
        
        # Create agent orchestrator
        agent_orchestrator = AgentOrchestrator(
            app_id=app_id,
            env=env,
            model_name=model_name,
            temperature=agent_temperature
        )
        agent_orchestrator.initialize(
            kg_retriever=kg_retriever,
            original_documents=extracted_text if extracted_text else "",
            vespa_wrapper=vespa_wrapper
        )
        
        # Create agent state
        agent_state = AgentState()
        
        tool_count = 10 if vespa_wrapper else 9
        print(f" ReAct Agent initialized successfully!")
        print(f" - Max iterations: {agent_max_iterations}")
        print(f" - Complexity threshold: {agent_complexity_threshold}/100")
        print(f" - {tool_count} specialized tools available\n")
    except ImportError as e:
        print(f" Agent modules not found: {str(e)}")
        print(" Using standard chat mode.\n")
        enable_agent_toggle.value = False
    except Exception as e:
        print(f" Could not initialize agent: {str(e)}")
        print(" Using standard chat mode.\n")
        enable_agent_toggle.value = False

# Chat history
chat_history = []

# Helper function to extract response
def _extract_response(response):
    """Extract actual content from LLM response."""
    if hasattr(response, 'content'):
        actual_response = response.content
    elif isinstance(response, dict):
        if 'Response' in response and 'content' in response['Response']:
            actual_response = response['Response']['content']
        elif 'content' in response:
            actual_response = response['content']
        else:
            actual_response = str(response)
    else:
        actual_response = str(response)
    
    # Clean up the response
    if actual_response:
        actual_response = actual_response.replace('\\n\\n', '\n\n')
        actual_response = actual_response.replace('\\n', '\n')
        actual_response = actual_response.strip()
    
    return actual_response

def chat_with_documents(question: str) -> str:
    """Send a question to the LLM and get a response with chat history."""
    
    # Determine if we should use agent
    use_agent_for_query = False
    routing_info = None
    
    if enable_agent_toggle.value and agent_orchestrator and query_router:
        use_agent_for_query, routing_info = query_router.should_use_agent(question)
        
    # Route to agent or simple flow
    if use_agent_for_query:
        # Use ReAct Agent
        print(f"  [Agent Mode] Complexity: {routing_info['complexity_score']}/100")
        
        agent_result = agent_orchestrator.query(
            query=question,
            include_trace=show_reasoning_toggle.value,
            state=agent_state
        )
        
        actual_response = agent_result.get('response', 'No response generated')
        
        # Add routing info if showing trace
        if show_reasoning_toggle.value and agent_result.get('trace'):
            trace_info = f"\n\n--- Agent Reasoning ---\n"
            trace_info += f"Query Type: {routing_info['query_type']}\n"
            trace_info += f"Complexity Score: {routing_info['complexity_score']}/100\n"
            trace_info += f"Iterations: {agent_result.get('iterations', 'N/A')}\n"
            trace_info += f"Routing Reason: {routing_info['routing_reason']}\n"
            actual_response += trace_info
            
    # Build enhanced prompt if KG is available (Simple Mode)
    elif use_kg and kg_retriever and extracted_text:
        print(f"  [Simple Mode] Using Knowledge Graph retrieval")
        full_prompt = kg_retriever.build_contextual_prompt(question, extracted_text)
        
        # Get response from LLM
        response = llm.invoke(full_prompt)
        actual_response = _extract_response(response)
    elif vespa_wrapper and not extracted_text:
        # Use Vespa when no documents uploaded
        print(f"  [Vespa Mode] Searching vector database")
        
        vespa_result = vespa_wrapper.search(question, top_k=10)
        vespa_context = vespa_wrapper.format_results_for_llm(vespa_result)
        
        full_prompt = f"""You are an assistant with access to a vector database.

{vespa_context}

Question: {question}

Instructions:
1. Use the search results above to answer the question
2. Cite specific results when providing answers
3. If results don't contain relevant information, state that clearly
4. Format your response professionally

Answer:"""
        
        response = llm.invoke(full_prompt)
        actual_response = _extract_response(response)
    else:
        # Use standard prompt (no KG, no agent, no Vespa)
        print(f"  [Standard Mode] Direct LLM query")
        full_prompt = f"""You are a cybersecurity and risk analysis assistant. Your role is to help users understand security controls, compliance requirements, risk assessments, and related governance documentation.

The documents may contain:
- Security controls and compliance frameworks
- Risk assessment data and audit findings
- Policy documents and governance standards
- Tables with control mappings, risk metrics, or compliance data
- Structured JSON/JSONL with control definitions, asset types, or security configurations
- Regular text describing security procedures and requirements

Document Content:
{extracted_text}

Question: {question}

Instructions:
1. Provide accurate, detailed answers based ONLY on the information in the provided documents
2. For security controls: Always include control IDs, names, and descriptions when available
3. For risk-related queries: Highlight severity, impact, likelihood, and mitigation measures
4. For compliance questions: Reference specific requirements, standards, and responsible parties
5. Format your response professionally with bullet points and clear organization
6. Always cite your sources precisely (e.g., "Table 2 on Page 5" or "JSON Object 3, control_id: 3997")
7. If information is missing, explicitly state what is available and what is not
8. Do not include raw JSON dumps - present information in a readable format
9. For questions about multiple controls or risks, organize your response systematically

Answer:"""
        
        # Get response from LLM
        response = llm.invoke(full_prompt)
        actual_response = _extract_response(response)
    
    # Store in chat history
    chat_history.append({
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "question": question,
        "response": actual_response
    })
    
    return actual_response

print("[SUCCESS] Interactive chat interface ready!")
if extracted_text:
    print(f"Documents loaded: {', '.join(document_names)}")
if use_kg and kg_retriever and extracted_text:
    print(" Knowledge Graph enhancement: ENABLED")
if vespa_wrapper:
    print(f" Vespa Vector Store: CONNECTED ({vespa_schema_input.value})")
if agent_orchestrator:
    tool_count = 10 if vespa_wrapper else 9
    print(f" ReAct Agent: ENABLED ({tool_count} tools available)")
    print(f" - Complexity threshold: {agent_complexity_threshold}/100")
    print(f" - Show reasoning: {'Yes' if show_reasoning_toggle.value else 'No'}")
print("\n" + "="*80)
if extracted_text:
    print("You can now chat with your documents!")
elif vespa_wrapper:
    print("You can now ask questions! (Using Vespa vector database)")
else:
    print("Upload documents or connect to Vespa to start chatting")
print("Type your questions and press Enter.")
print("Type 'exit()' to stop chatting.")
print("Type 'history' to view chat history.")
if agent_orchestrator:
    print("\nAgent will automatically activate for complex queries!")
print("="*80 + "\n")


## 8. Start Continuous Chat

Run this cell to start chatting with your documents. You can ask unlimited questions until you type 'exit()'.


In [ ]:
# Continuous chat loop - Run this cell to start chatting
if 'chat_with_documents' not in globals():
    print("[WARNING] Please run section 7 first to initialize the chat interface")
elif extracted_text is None and not vespa_wrapper:
    print("[WARNING] Please extract text from documents (section 4) OR connect to Vespa (section 7)")
else:
    print(" Chat started! Ask your questions below.\n")
    
    # Continuous chat loop
    while True:
        try:
            # Get user input
            question = input("\n You: ").strip()
            
            # Check for exit command
            if question.lower() in ['exit()', 'exit', 'quit', 'quit()']:
                print("\n Exiting chat. Thank you!")
                print(f"Total questions asked: {len(chat_history)}")
                break
            
            # Check for history command
            if question.lower() == 'history':
                if chat_history:
                    print("\n Chat History:")
                    print("="*80)
                    for i, entry in enumerate(chat_history, 1):
                        print(f"\n[{i}] {entry['timestamp']}")
                        print(f"Q: {entry['question']}")
                        print(f"A: {entry['response'][:200]}..." if len(entry['response']) > 200 else f"A: {entry['response']}")
                        print("-"*80)
                else:
                    print("\n No chat history yet.")
                continue
            
            # Skip empty questions
            if not question:
                print(" Please enter a question.")
                continue
            
            # Get response
            print("\n Thinking...")
            response = chat_with_documents(question)
            
            # Display response
            print("\n Assistant:")
            print("="*80)
            print(response)
            print("="*80)
            
        except KeyboardInterrupt:
            print("\n\n Chat interrupted. Type 'exit()' to quit properly.")
            break
        except EOFError:
            print("\n\n Exiting chat. Thank you!")
            print(f"Total questions asked: {len(chat_history)}")
            break
        except Exception as e:
            print(f"\n Error: {str(e)}")
            print("Please try again or type 'exit()' to quit.")
    
    print(f"\n Session Summary:")
    print(f"  - Questions asked: {len(chat_history)}")
    print(f"  - Documents processed: {len(document_names)}")
    if use_kg and kg_retriever:
        stats = kg_retriever.get_statistics()
        print(f"  - Entities in KG: {stats['entity_count']}")
        print(f"  - Relationships in KG: {stats['relationship_count']}")
    if agent_state:
        agent_stats = agent_state.get_statistics()
        print(f"  - Agent tool calls: {agent_stats['tool_calls_made']}")
        print(f"  - Entities discovered: {agent_stats['entities_discovered']}")


## 9. View Full Chat History (Optional)

Run this cell anytime to view the complete chat history in a formatted table.


In [ ]:
# Display full chat history as a formatted table
if 'chat_history' not in globals():
    print(" Please run section 7 first to initialize the chat interface")
elif not chat_history:
    print(" No chat history yet. Start asking questions in section 8!")
else:
    print(f" Chat History ({len(chat_history)} questions)\n")
    print("="*100)
    
    for i, entry in enumerate(chat_history, 1):
        print(f"\n[Question {i}] {entry['timestamp']}")
        print(f"{'─'*100}")
        print(f" Question: {entry['question']}")
        print(f"\n Answer:")
        print(entry['response'])
        print("="*100)
    
    # Also display as DataFrame for easy export
    print("\n Exportable Table View:\n")
    df_history = pd.DataFrame(chat_history)
    display(df_history)
    
    # Statistics
    print(f"\n Statistics:")
    print(f"  - Total questions: {len(chat_history)}")
    print(f"  - Average response length: {sum(len(e['response']) for e in chat_history) / len(chat_history):.0f} characters")
    
    if use_kg and kg_retriever:
        stats = kg_retriever.get_statistics()
        print(f"  - Knowledge Graph entities: {stats['entity_count']}")
        print(f"  - Knowledge Graph relationships: {stats['relationship_count']}")
